# Jass Agent with Reinforcement Learning
### by Ruben Nunez & Jordan Suter

This is highly inspired by the paper of Jacop Chapman and Mathias Lechner. <<Deep Q-Learning for Atari Breakout>>

We tried to implement the Jass-Agent similarly as described in the paper.


<figure>
    <img src="img/reinforcement.png" width="600"/>
</figure>

### table of contents:
* Setup
* Parameter
* Deep Q-Network
* Agent
* Environment
* Training (Jass loop)

### source directory
- https://de.mathworks.com/discovery/reinforcement-learning.html
- https://keras.io/examples/rl/deep_q_network_breakout/
- https://www.analyticsvidhya.com/blog/2019/04/introduction-deep-q-learning-python/
- https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html#sphx-glr-auto-examples-classification-plot-classifier-comparison-py

UNO BOT mit sequential Model
- https://github.com/PhilippThoelke/uno-bot/blob/master/agent.py
- https://www.tensorflow.org/guide/keras/save_and_serialize

## Setup

In [1]:
import sys
import os

import numpy as np
import pandas as pd
import tensorflow as tf

from jass.game.game_util import *
from jass.game.game_sim import GameSim
from jass.game.game_observation import GameObservation
from jass.game.const import *
from jass.game.rule_schieber import RuleSchieber
from jass.agents.agent import Agent
from jass.agents.agent_random_schieber import AgentRandomSchieber
from jass.arena.arena import Arena

from tensorflow import keras
from tensorflow.keras import layers, models

sys.path.append("../deep-jass-project")
# noinspection PyUnresolvedReferences
from agent_gen1 import AgentGen1
# noinspection PyUnresolvedReferences
from agent_gen2 import AgentGen2
# noinspection PyUnresolvedReferences
from agent_gen3 import AgentGen3
# noinspection PyUnresolvedReferences
from agent_gen4 import AgentGen4
# noinspection PyUnresolvedReferences
from agent_gen5 import AgentGen5
# noinspection PyUnresolvedReferences
from agent_intelligent import AgentIntelligent
# noinspection PyUnresolvedReferences
from agent_helper import get_remaining_cards

## Parameter

In [2]:
seed = 42
gamma = 0.99  # Discount factor for past rewards
epsilon = 1.0  # Epsilon greedy parameter
epsilon_min = 0.1  # Minimum epsilon greedy parameter
epsilon_max = 1.0  # Maximum epsilon greedy parameter
epsilon_interval = (epsilon_max - epsilon_min)
batch_size = 36  # Size of batch taken from replay buffer
max_steps_per_episode = 36 # ein Spiel geht maximal 36 Züge lang

# In the Deepmind paper they use RMSProp however then Adam optimizer
optimizer = keras.optimizers.Adam(learning_rate=0.00025, clipnorm=1.0) # improves training time
model_path = 'model/model_current'

# Experience replay buffers
action_history = []
state_history = []
state_next_history = []
rewards_history = []
done_history = []
episode_reward_history = []
running_reward = 0
episode_count = 0
trick_count = 0

epsilon_random_frames = 100  # Number of frames to take random action and observe output
epsilon_greedy_frames = 1000.0  # Number of frames for exploration

# Note: The Deepmind paper suggests 1000000 however this causes memory issues
max_memory_length = 360 # Maximum replay length
update_after_actions = 1  # Train the model after 1 actions
update_target_network = 360  # How often to update the target network
loss_function = keras.losses.Huber()  # Using huber loss for stability

## Deep Q-Network
This network learns an approximation of the Q-table, which is a mapping between the states and actions that an agent will take.
In every state are 36 actions, that can eventually be taken. The environment provides the state, and the action is chosen by selecting the larger of the 36 Q-values predicted in the output layer.

In [3]:
num_actions = 36

def create_UNO_model(input_size, output_size):

    if os.path.exists(model_path):
        _model = keras.models.load_model(model_path)
        return _model

    # define the model architecture
    _model = models.Sequential()
    _model.add(layers.Dense(units=64, activation='relu', input_shape=(input_size,)))
    _model.add(layers.Dense(units=64, activation='relu'))
    _model.add(layers.Dense(units=64, activation='relu'))
    _model.add(layers.Dense(units=64, activation='relu'))
    _model.add(layers.Dense(units=output_size, activation='linear'))

    # compile the model
    _model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
    return _model


# The first model makes the predictions for Q-values which are used to
# make a action.
model = create_UNO_model(160, 36)
# Build a target model for the prediction of future rewards.
# The weights of a target model get updated every 10000 steps thus when the
# loss between the Q-values is calculated the target Q-value is stable.
model_target = create_UNO_model(160, 36)

2021-12-13 14:59:47.879018: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Agent

In [10]:
# Reinforcement Agent Implementation
# by Ruben Nunez & Jordan Suter

class AgentReinforcement(Agent):

    agent = AgentIntelligent()

    def __init__(self):
        super().__init__()
        self._rule = RuleSchieber() # we need a rule object to determine the valid cards

    def action_trump(self, obs: GameObservation) -> int:
        return self.agent.action_trump(obs)

    def action_play_card(self, obs: GameObservation, _action) -> int:
        valid_cards = self._rule.get_valid_cards_from_obs(obs)
        if valid_cards[_action] == 1:
            card = _action
        else:
            card = np.random.choice(np.flatnonzero(valid_cards))

        return card

## Environment

In [11]:
class Environment:

    game = None
    rule = None

    agent_reinforcement = None
    agent_other = None

    def __init__(self):
        self.agent_reinforcement = AgentReinforcement()
        self.agent_other = AgentIntelligent()
        self.rule = RuleSchieber()

    def init_agent(self):
        self.agent_other = AgentIntelligent()

    def new_game(self):
        game = GameSim(rule=self.rule)

        # deal cards
        game.init_from_cards(hands=deal_random_hand(), dealer=SOUTH)
        obs = game.get_observation()

        trump = self.agent_reinforcement.action_trump(obs) # set trump
        game.action_trump(trump)

        if trump == PUSH: # set PUSH
            obs = game.get_observation()
            trump = self.agent_2.action_trump(obs) # set trump
            game.action_trump(trump) # tell the simulation the selected trump

        self.game = game

    def step(self, _action):
        obs = self.game.get_observation()
        _reward = 0

        if obs.player == 0:
            valid_cards = self.rule.get_valid_cards_from_obs(obs)
            card = None
            if valid_cards[_action] == 1:
                card = _action
                _reward += 100
            else:
                card = np.random.choice(np.flatnonzero(valid_cards))
                _reward += -100
            self.game.action_play_card(card)
            _reward += 5 * (obs.points[0] - obs.points[1])
        else:
            self.game.action_play_card(self.agent_other.action_play_card(obs))

        _done = self.game.is_done()
        _state = self.state(),

        return _reward, _done


    def state(self):
        obs = self.game.get_observation()
        if obs.current_trick is None:
            obs.current_trick = np.array([-1, -1, -1, -1])

        remaining_cards = get_remaining_cards(obs)
        valid_cards = self.game.rule.get_valid_cards_from_obs(obs)

        trump = np.zeros((10,))
        trump[obs.declared_trump] = 1
        result = np.concatenate((obs.hand, valid_cards, remaining_cards, obs.tricks.reshape(36), obs.current_trick, trump, np.array([obs.nr_cards_in_trick]),  np.array([obs.nr_played_cards])))

        return result


## Training (Jass loop)

In [13]:
env = Environment()

# In the Deepmind paper they use RMSProp however then Adam optimizer
# improves training time
optimizer = keras.optimizers.Adam(learning_rate=0.00025, clipnorm=1.0)

# Experience replay buffers
action_history = []
state_history = []
state_next_history = []
rewards_history = []
done_history = []
episode_reward_history = []
running_reward = 0
episode_count = 1000000
move_count = 0

# Maximum replay length
# Note: The Deepmind paper suggests 1000000 however this causes memory issues
max_memory_length = 100000
update_after_actions = 4 # Train the model after 4 actions
update_target_network = 1000 # How often to update the target network
loss_function = keras.losses.Huber() # Using huber loss for stability


while True:  # Run until solved
    env.new_game() # Start Game (80,)
    state = env.state()
    episode_reward = 0

    for step in range(0, max_steps_per_episode):

        # of the agent in a pop up window.
        move_count += 1

        # Use epsilon-greedy for exploration
        if move_count < epsilon_random_frames or epsilon > np.random.rand(1)[0]:
            # Take random action
            action = np.random.choice(num_actions)
        else:
            # Predict action Q-values
            # From environment state
            state_tensor = tf.convert_to_tensor(state)
            state_tensor = tf.expand_dims(state_tensor, 0)
            action_probs = model(state_tensor, training=False)
            # Take best action
            action = tf.argmax(action_probs[0]).numpy()

        # Decay probability of taking random action
        epsilon -= epsilon_interval / epsilon_greedy_frames
        epsilon = max(epsilon, epsilon_min)

        # Apply the sampled action in our environment
        reward, done = env.step(action)
        state_next = env.state()
        # state_next = np.array(state_next)

        episode_reward += reward

        # Save actions and states in replay buffer
        action_history.append(action)
        state_history.append(state)
        state_next_history.append(state_next)
        done_history.append(done)
        rewards_history.append(reward)
        state = state_next

        # Update every fourth frame and once batch size is over 32
        if move_count % update_after_actions == 0 and len(done_history) > batch_size:

            # Get indices of samples for replay buffers
            indices = np.random.choice(range(len(done_history)), size=batch_size)

            # Using list comprehension to sample from replay buffer
            state_sample = np.array([state_history[i] for i in indices])
            state_next_sample = np.array([state_next_history[i] for i in indices])
            rewards_sample = [rewards_history[i] for i in indices]
            action_sample = [action_history[i] for i in indices]
            done_sample = tf.convert_to_tensor(
                [float(done_history[i]) for i in indices]
            )

            # Build the updated Q-values for the sampled future states
            # Use the target model for stability

            future_rewards = model_target.predict(state_next_sample)
            # Q value = reward + discount factor * expected future reward
            updated_q_values = rewards_sample + gamma * tf.reduce_max(
                future_rewards, axis=1
            )

            # If final frame set the last value to -1
            updated_q_values = updated_q_values * (1 - done_sample) - done_sample

            # Create a mask so we only calculate loss on the updated Q-values
            masks = tf.one_hot(action_sample, num_actions)

            with tf.GradientTape() as tape:
                # Train the model on the states and updated Q-values
                q_values = model(state_sample)

                # Apply the masks to the Q-values to get the Q-value for action taken
                q_action = tf.reduce_sum(tf.multiply(q_values, masks), axis=1)
                # Calculate loss between new Q-value and old Q-value
                loss = loss_function(updated_q_values, q_action)

            # Backpropagation
            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if move_count % update_target_network == 0:
            # update the the target network with new weights
            model_target.set_weights(model.get_weights())
            # Log details
            template = "running reward: {:.2f} at episode {}, frame count {}"
            print(template.format(running_reward, episode_count, move_count))

        # Limit the state and reward history
        if len(rewards_history) > max_memory_length:
            del rewards_history[:1]
            del state_history[:1]
            del state_next_history[:1]
            del action_history[:1]
            del done_history[:1]

        if done:
            a = 0
            break

    # Update running reward to check condition for solving
    episode_reward_history.append(episode_reward)
    if len(episode_reward_history) > 100:
        del episode_reward_history[:1]
    running_reward = np.mean(episode_reward_history)

    episode_count += 1

    if episode_count % 100 == 0:
        model.save(model_path)
        env.init_agent()

    if episode_count % 10000 == 0:
        model.save('model/model_' + str(episode_count))

    from IPython.display import clear_output, display
    clear_output(wait=True)
    print("difference: " + str(env.game.get_observation().points[0] - env.game.get_observation().points[1]) + " @Episode: " + str(episode_count) + " running: reward = "  + str(running_reward), flush=True)

    if running_reward > 40000:  # Condition to consider the task solved
        print("Solved at episode {}!".format(episode_count))
        break

difference: -81 @Episode: 1006347 running: reward = -1630.0


KeyboardInterrupt: 